### Downloading Dataset 

In [1]:
import os 
from tqdm import tqdm
#from google.colab import drive
import shutil 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import matplotlib.pyplot  as plt
from pathlib import Path
import imagesize
import numpy as np
import torch
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
import pandas as pd
import zipfile 
import statistics 


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!mkdir dataset
!cp "gdrive/My Drive/DLL_project/dataset/Adaptiope.zip" dataset/
!ls dataset

Adaptiope.zip


In [ ]:
!unzip dataset/Adaptiope.zip

Streaming output truncated to the last 5000 lines.
  inflating: Adaptiope/synthetic/purse/purse_1_19.png  
  inflating: Adaptiope/synthetic/purse/purse_2_16.png  
  inflating: Adaptiope/synthetic/purse/purse_4_16.png  
  inflating: Adaptiope/synthetic/purse/purse_3_03.png  
  inflating: Adaptiope/synthetic/purse/purse_3_16.png  
  inflating: Adaptiope/synthetic/purse/purse_1_17.png  
  inflating: Adaptiope/synthetic/purse/purse_2_05.png  
  inflating: Adaptiope/synthetic/purse/purse_3_08.png  
  inflating: Adaptiope/synthetic/purse/purse_2_07.png  
  inflating: Adaptiope/synthetic/purse/purse_2_17.png  
  inflating: Adaptiope/synthetic/purse/purse_3_00.png  
  inflating: Adaptiope/synthetic/purse/purse_1_09.png  
  inflating: Adaptiope/synthetic/purse/purse_4_18.png  
  inflating: Adaptiope/synthetic/purse/purse_3_05.png  
  inflating: Adaptiope/synthetic/purse/purse_3_01.png  
  inflating: Adaptiope/synthetic/purse/purse_4_08.png  
  inflating: Adaptiope/synthetic/purse/purse_4_09.png

In [ ]:
! rm -rf adaptiope_small

In [ ]:
! mkdir adaptiope_small

In [7]:
classes = ["backpack", "bookcase", "car jack", "comb", "crown", "file cabinet", "flat iron", "game controller", "glasses",
           "helicopter", "ice skates", "letter tray", "monitor", "mug", "network switch", "over-ear headphones", "pen",
           "purse", "stand mixer", "stroller"]

domains = ["product_images", "real_life"]




In [ ]:
with zipfile.ZipFile("data/Adaptiope.zip", 'r') as zip_f:
    namelist = zip_f.namelist() 
    images = [x for x in namelist if x.endswith(".jpg")]
    filter = [f"Adaptiope/{y}/{x}/" for y in domains for x in classes]
    cl_images = [x for x in images if x.startswith(tuple(filter))]
    zip_f.extractall(path = "data/", members= cl_images)

## EDA: Exploratory Data Analysis


In [8]:

metadata_img = {}

for domain in domains:
  #print(domain)
  subclass = os.listdir(f"data/Adaptiope/{domain}")
  for class_name in classes:
    imgs = [img.name for img in Path(f"data/Adaptiope/{domain}/{class_name}/").iterdir() if img.suffix == ".jpg"]
    for fl in imgs: 
      metadata_img[str(fl + domain)] = imagesize.get(f"data/Adaptiope/{domain}/{class_name}/"+fl)



In [12]:


# some statistics to check how many images we have and their dimensions (on average)
img_df = pd.DataFrame.from_dict([metadata_img]).T.reset_index().set_axis(['file_name', 'size'], axis='columns', inplace=False)
img_df[["width", "height"]] = pd.DataFrame(img_df["size"].tolist(), index=img_df.index)
avg_width = round(img_df["width"].sum() / len(img_df))
avg_height = round(img_df["height"].sum() / len(img_df))
sd_width = round(statistics.stdev(img_df["width"]), 2)
sd_height = round(statistics.stdev(img_df["height"]), 2)


print(f'Total images in Adaptiope dataset: {len(img_df)}')
print(f'Average height in Adaptiope dataset: {avg_height}')
print(f'SD height in Adaptiope dataset: {sd_height}')

print(f'Average width in Adaptiope dataset: {avg_width}')
print(f'SD width in Adaptiope dataset: {sd_width}')


Total images in Adaptiope dataset: 4000
Average height in Adaptiope dataset: 1122
SD height in Adaptiope dataset: 415.62
Average width in Adaptiope dataset: 1160
SD width in Adaptiope dataset: 415.42


In [15]:
thre = 1e-5
cond = np.logical_and(img_df['height'] - thre < img_df['width'], img_df['width'] < img_df['height'] + thre)
sq_df = img_df[cond]
print('Percentage square images:', len(sq_df.index)/len(img_df.index))

Percentage square images: 0.323


In [17]:
rect_df = img_df[~cond]
avg_size_diff = round(sum(abs(rect_df['width'] - rect_df['height'])) / len(rect_df.index), 2)
std_size_diff = round(statistics.stdev(abs(rect_df['width'] - rect_df['height'])), 2)
print(f'Size difference = {avg_size_diff} +/- {std_size_diff}')

Size difference = 421.57 +/- 258.17


In [19]:
print("Width:", "from", min(img_df["width"]), "to", max(img_df["width"]))
print("Height", "from", min(img_df["height"]), "to", max(img_df["height"]))

Width: from 120 to 2870
Height from 75 to 2560


## Data Transformation

In [20]:
def data_transformation_product(m = 256, n = 224):
  """
  Data transformation for training data

  Args:
      m (int, optional): _description_. Defaults to 256.
      n (int, optional): _description_. Defaults to 224.

  Returns:
      _type_: _description_
  """
  
  transform_adaptiope_product = []
  transform_adaptiope_product.append(T.Resize((m)))                                          # resize each PIL image to m x m                  
  transform_adaptiope_product.append(T.Grayscale(num_output_channels=1))                        # to make RGB to grayscale
  transform_adaptiope_product.append(T.RandomCrop((n)))                                       # randomly crop a n x n patch
  transform_adaptiope_product.append(T.RandomHorizontalFlip(p=0.5))                                  
  transform_adaptiope_product.append(T.ToTensor())                                              
  #transform_adaptiope_product.append(T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]))    
  return T.Compose(transform_adaptiope_product)  


def data_transformation_real(m = 256, n = 224):
  """
  Data Transformation for test data

  Args:
      m (int, optional): _description_. Defaults to 256.
      n (int, optional): _description_. Defaults to 224.

  Returns:
      _type_: _description_
  """
  
  transform_adaptiope_real = []
  transform_adaptiope_real.append(T.Resize((m)))                                          # resize each PIL image to m x m                  
  transform_adaptiope_real.append(T.Grayscale(num_output_channels=1))                        # to make RGB to grayscale
  transform_adaptiope_real.append(T.CenterCrop(n))                                    # randomly crop a n x n patch
  transform_adaptiope_real.append(T.RandomHorizontalFlip(p=0.5))                                  
  transform_adaptiope_real.append(T.ToTensor())                                              
  
  return T.Compose(transform_adaptiope_real)    


In [ ]:
# TODO: implementare con sklearn lo split 

from sklearn.model_selection import StratifiedShuffleSplit

def get_data(batch_size, source = "product_images", target = "real_life"):     
                              
  # TODO: estrazione in data target e data source 
  # TODO: shuffle split (salvataggio cartella)
  # TODO: transform 
  # TODO: data loader 
  
  # load source and split train and test 
  source_data = torchvision.datasets.ImageFolder(root=f"data/Adaptiope/{source}")

  split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

  source_training_samples, source_test_samples = split.split(source_data, )
  
  source_training_data, source_test_data = torch.utils.data.random_split(source_data, 
                                                           [source_training_samples, source_test_samples])


  # load target and split in train and test 
  target_data = torchvision.datasets.ImageFolder(root=f"data/Adaptiope/{target}", transform=data_transformation())


  target_training_samples, target_test_samples = split_test_train(target_data)
  target_training_data, target_test_data = torch.utils.data.random_split(target_data, 
                                                           [target_training_samples, target_test_samples])

  # initialize dataloaders
  source_train_loader = torch.utils.data.DataLoader(source_training_data, batch_size, shuffle=True, drop_last=True)
  source_test_loader = torch.utils.data.DataLoader(source_test_data, batch_size, shuffle=False)
  
  target_train_loader = torch.utils.data.DataLoader(target_training_data, batch_size, shuffle=True, drop_last=True)
  target_test_loader = torch.utils.data.DataLoader(target_test_data, batch_size, shuffle=False)
  
  return source_train_loader, source_test_loader, target_train_loader, target_test_loader 




# Feature extraction 

- 

In [1]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

NameError: ignored

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

In [ ]:

for param in model.parameters():
    param.requires_grad = False

In [ ]:
features = model.fc.in_features

model.fc = nn.Linear(features, len(dataset.classes))
